# Astropy: Celestial Coordinates
This notebook is heavily based on one from Thomas Robitaille

## Documentation

For more information about the features presented below, you can read the
[astropy.coordinates](http://docs.astropy.org/en/stable/coordinates/index.html) docs.

## Representing and converting coordinates

Astropy includes a framework to represent celestial coordinates and transform
between them. Astropy includes common coordinate systems (ICRS (Ra and Dec), Galactic, and AltAz).

Creating coordinate objects is straightforward:

In [ ]:
import astropy.coordinates as ac
import astropy.units as au

In [ ]:
# different ways to represent Ra and Dec coordinates
obj_1 = ac.SkyCoord(ra=10.68458 * au.deg, dec=41.26917 * au.deg)
obj_2 = ac.SkyCoord(ra='00:42:44.3', dec='+41:16:09', unit=(au.hourangle, au.deg))
obj_3 = ac.SkyCoord('00h42m44.3s +41d16m9s')

print(obj_1, obj_2, obj_3)

The individual components of a coordinate are ``Angle`` objects, and their
values are accessed using special attributes:

In [ ]:
print(obj_1.ra)
print(obj_1.ra.hour)
print(obj_1.ra.hms)
print(obj_1.dec)
print(obj_1.dec.radian)

To convert to some other coordinate system, the easiest method is to use
attribute-style access with short names for the built-in systems:

In [ ]:
obj_1.galactic

but explicit transformations via the ``transform_to`` method are also available:

In [ ]:
obj_1.transform_to('galactic')

The ``astropy.coordinates`` subpackage also provides a quick way to get coordinates for
named objects (with an internet connection). The SkyCoord class has a method `from_name()`, that accepts a string and queries [Sesame](http://cds.u-strasbg.fr/cgi-bin/Sesame) to retrieve coordinates for that
object:

In [ ]:
# obtain coordinates from the cluster Abell 1689 over the internet
a1689_coord = ac.SkyCoord.from_name("A1689")
a1689_coord

## Using arrays in coordinates

Numpy arrays and table columns can be used inside coordinate objects instead of scalar floating point values (this is much more efficient that creating one coordinate object for each source). The following example demonstrates how one can combine the ``Table`` class with coordinate objects.

In [ ]:
# obtain coordinates of open clusters fro the Lab-course:
import astropy.io.ascii as aia

open_clusters = aia.read('data/open_clusters.txt')
open_clusters

In [ ]:
cluster_coord = ac.SkyCoord(open_clusters['RA'], open_clusters['DEC'], unit=(au.hourangle, au.deg))

## Converting to/from AltAz - Planning Lab observations of NGC581

For your LAB-course (photometry of star clusters), you had to plan your observations and to create plots as the following (visibility of your objects):
<img src="figs/obs_ngc6913.png" style="width: 500px">

In [ ]:
import numpy as np
import astropy.units as au
import astropy.time as at
import astropy.coordinates as ac

ngc6913 = ac.SkyCoord.from_name('NGC6913')
ngc6913

In [ ]:
# location of AIfA (taken from the LAB manual)
AIfA = ac.EarthLocation(lon='07d04m01s', lat='50d43m46s', height=75 * au.m)
utcoffset = +2 * au.hour  # time offset w.r.t. UTC (note: summer time!)
time = at.Time('2018-07-11T00:00:00') - utcoffset

In [ ]:
ngc6913altaz = ngc6913.transform_to(ac.AltAz(obstime=time,location=AIfA))

In [ ]:
ngc6913altaz.alt

We now want to make a plot of the altitude vs time to plan our observations

In [ ]:
midnight = at.Time('2018-07-11T00:00:00') - utcoffset
delta_midnight = np.linspace(-4, 7, 100) * au.hour

In [ ]:
ngc6913altazs = ngc6913.transform_to(ac.AltAz(obstime=midnight+delta_midnight, location=AIfA))  

We can now plot the results:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(delta_midnight, ngc6913altazs.alt)  
plt.xlim(-4, 7)  
plt.ylim(10, 90)
#plt.ylim(1.5, 1.0)
plt.axhline(0, color='k', ls='dashed')
plt.xlabel('Hours from Midnight')  
plt.ylabel('object elevation over the horizon [deg]')  
plt.show()